<a href="https://colab.research.google.com/github/hirohitokato/ohm_princess_pytorch/blob/main/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Pythonで機械学習入門機械学習入門(Pytorch版) **Chapter2**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim	# 最適化関数関係(from chainer import optimizers)
import torch.nn.functional as F	# 関数関係(import chainer.functions as F)
import torch.nn as nn	# ネットワーク関係(import chainer.links as L)

`from chainer import Variable, Chain, config` については以下のように扱う。。
* `Variable(np.ndarray)` : `torch.from_numpy(np.ndarray)`
* `class DNN(Chain):` : `class DNN(nn.Module):`
* `config.train = True` : **不明。要調査**

[公式のmigration_guide](https://chainer.github.io/migration-guide/)には次のように書いてある。

> Explains differences of how train/test modes are switched.
> * Chainer: This mode is controlled via a configuration (configuration.config.train).
> * PyTorch: This mode is a module state and should be changed using [torch.nn.Module.train(mode)](https://www.google.com/url?q=https://pytorch.org/docs/stable/nn.html%23torch.nn.Module.train&sa=D&source=editors&ust=1615451627944000&usg=AOvVaw1LtL0CCtZGBEHSHWobAyOG), [torch.nn.Module.eval()](https://www.google.com/url?q=https://pytorch.org/docs/stable/nn.html%23torch.nn.Module.eval&sa=D&source=editors&ust=1615451627944000&usg=AOvVaw1zTZQi3CLT-abAXb4xfMZp). This also means that you must pass the “train” argument to functions calls such as [torch.nn.functional.dropout](https://www.google.com/url?q=https://pytorch.org/docs/stable/nn.functional.html%23torch.nn.functional.dropout&sa=D&source=editors&ust=1615451627944000&usg=AOvVaw3WbrBYZ7GIbYZ3YU-g-QHz) otherwise use the [torch.nn.Dropout module](https://www.google.com/url?q=https://pytorch.org/docs/stable/nn.html%23torch.nn.Dropout&sa=D&source=editors&ust=1615451627945000&usg=AOvVaw17jwofkuPJvc55crtYQATG) which is stateful.
> * In both Chainer and PyTorch, train/test mode affects the behavior of certain functions/links/modules such as dropout and batch normalization.
> * Both Chainer and PyTorch default to train mode.

In [ ]:
D = 100
N = 2
xdata = np.random.randn(D*N).reshape(D,N).astype(np.float32)